In [12]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import pickle
import Plot
import model
import numpy as np
import pandas as pd

# Get lambdas

In [13]:
def generate_lambdas(size):
    weights = []
    # 36 weight vectors
    for i in range(1,9):
        for j in range(i+1,10):
            weights.append([i,j-i,10-j])
    res = []
    for i in range(int(size/2)):
        res.append(weights[i])
        res.append(weights[-i-1])
    res = np.array(res)
    indx = np.lexsort((res[:,2],res[:,0],res[:,1]))
    return (res[indx])/10

lambdas = generate_lambdas(20)

In [14]:
lambdas

array([[0.1, 0.1, 0.8],
       [0.2, 0.1, 0.7],
       [0.5, 0.1, 0.4],
       [0.6, 0.1, 0.3],
       [0.7, 0.1, 0.2],
       [0.8, 0.1, 0.1],
       [0.1, 0.2, 0.7],
       [0.2, 0.2, 0.6],
       [0.5, 0.2, 0.3],
       [0.6, 0.2, 0.2],
       [0.7, 0.2, 0.1],
       [0.1, 0.3, 0.6],
       [0.5, 0.3, 0.2],
       [0.6, 0.3, 0.1],
       [0.1, 0.4, 0.5],
       [0.5, 0.4, 0.1],
       [0.1, 0.5, 0.4],
       [0.1, 0.6, 0.3],
       [0.1, 0.7, 0.2],
       [0.1, 0.8, 0.1]])

# Compute Objective functions

In [15]:
def compute_objective_functions(solution, sensors_positions, sink_nodes_positions):
    f = [0,0,0] 
    for i in range(len(solution)):
        if(solution[i][0]==1):
            f[0] += solution[i][1]**2
            f[1] += 1

            distance = np.sqrt((sensors_positions[i][0]-sink_nodes_positions[0][0])**2 + (sensors_positions[i][1]-sink_nodes_positions[0][1])**2)
        
            f[2] += distance
            
    f[2]/=f[1]
    return f

In [16]:
# Contains first solution of every run times of every datasets
df_first = pd.DataFrame(columns=['Dataset No.','Run No.','Power weight','Num. weight','Dist. weight','Power','Num. Active Sensors','Avg. Dist. to Sink node'])
# Contains last solution of every run times of every datasets
df_last = pd.DataFrame(columns=['Dataset No.','Run No.','Power weight','Num. weight','Dist. weight','Power','Num. Active Sensors','Avg. Dist. to Sink node'])

In [17]:
num_dataset = 5
for dataset in range(num_dataset):
    with open(f'Datasets/uniform/50x1000unit/100sensors/sensors_positions_{dataset}.pickle','rb') as file:
        sensors_positions = pickle.load(file)
    with open('Datasets/sink_nodes_positions.pickle','rb') as file:
        sink_nodes_positions = pickle.load(file) 

    for runtime in range(5):
        with open(f'Results/uniform/50x1000unit/100sensors/dataset_{dataset}/last_solutions_{runtime}.pickle','rb') as file:
            last_sol = pickle.load(file)
        with open(f'Results/uniform/50x1000unit/100sensors/dataset_{dataset}/first_solutions_{runtime}.pickle','rb') as file:
            first_sol = pickle.load(file)
        
        for i in range(len(first_sol)):
            first_sol_f = compute_objective_functions(first_sol[i], sensors_positions, sink_nodes_positions)
            last_sol_f = compute_objective_functions(last_sol[i], sensors_positions, sink_nodes_positions)
            df_first.loc[len(df_first)] = [dataset, runtime, lambdas[i][0],lambdas[i][1],lambdas[i][2], first_sol_f[0], first_sol_f[1], first_sol_f[2]]
            df_last.loc[len(df_last)] = [dataset, runtime, lambdas[i][0],lambdas[i][1],lambdas[i][2], last_sol_f[0], last_sol_f[1], last_sol_f[2]]

In [18]:
df_first.iloc[-20:]

,Dataset No.,Run No.,Power weight,Num. weight,Dist. weight,Power,Num. Active Sensors,Avg. Dist. to Sink node
480,4.0,4.0,0.1,0.1,0.8,18652.00239,28.0,749.266806
481,4.0,4.0,0.2,0.1,0.7,18652.00239,28.0,749.266806
482,4.0,4.0,0.5,0.1,0.4,18652.00239,28.0,749.266806
483,4.0,4.0,0.6,0.1,0.3,18652.00239,28.0,749.266806
484,4.0,4.0,0.7,0.1,0.2,18652.00239,28.0,749.266806
485,4.0,4.0,0.8,0.1,0.1,18652.00239,28.0,749.266806
486,4.0,4.0,0.1,0.2,0.7,18652.00239,28.0,749.266806
487,4.0,4.0,0.2,0.2,0.6,18652.00239,28.0,749.266806
488,4.0,4.0,0.5,0.2,0.3,18652.00239,28.0,749.266806
489,4.0,4.0,0.6,0.2,0.2,18652.00239,28.0,749.266806


In [19]:
df_last.iloc[-20:].sort_values(by=['Num. Active Sensors'])

,Dataset No.,Run No.,Power weight,Num. weight,Dist. weight,Power,Num. Active Sensors,Avg. Dist. to Sink node
480,4.0,4.0,0.1,0.1,0.8,42247.223467,15.0,621.582271
486,4.0,4.0,0.1,0.2,0.7,42247.223467,15.0,621.582271
491,4.0,4.0,0.1,0.3,0.6,35073.975870,17.0,624.387714
494,4.0,4.0,0.1,0.4,0.5,30180.863540,18.0,626.263802
499,4.0,4.0,0.1,0.8,0.1,15897.796564,19.0,746.911646
498,4.0,4.0,0.1,0.7,0.2,14640.719436,21.0,749.623549
496,4.0,4.0,0.1,0.5,0.4,19706.793565,22.0,659.635524
497,4.0,4.0,0.1,0.6,0.3,19706.793565,22.0,659.635524
487,4.0,4.0,0.2,0.2,0.6,19750.078480,24.0,680.655944
481,4.0,4.0,0.2,0.1,0.7,19750.078480,24.0,680.655944


# Join table by Dataset No., Run No., Lambdas

In [20]:
df_join =pd.merge(df_first,df_last,on=['Dataset No.','Run No.','Power weight','Num. weight','Dist. weight'],how='outer', suffixes=('_first','_last'))

In [21]:
df_join.to_csv('Results/uniform/50x1000unit/100sensors/First_vs_Last_solutions.csv')

In [22]:
df_join

,Dataset No.,Run No.,Power weight,Num. weight,Dist. weight,Power_first,Num. Active Sensors_first,Avg. Dist. to Sink node_first,Power_last,Num. Active Sensors_last,Avg. Dist. to Sink node_last
0,0.0,0.0,0.1,0.1,0.8,14430.53002,34.0,734.913664,44496.889618,16.0,603.935239
1,0.0,0.0,0.2,0.1,0.7,14430.53002,34.0,734.913664,19745.516547,25.0,647.457872
2,0.0,0.0,0.5,0.1,0.4,14430.53002,34.0,734.913664,7902.701230,42.0,753.847585
3,0.0,0.0,0.6,0.1,0.3,14430.53002,34.0,734.913664,7902.701230,42.0,753.847585
4,0.0,0.0,0.7,0.1,0.2,14430.53002,34.0,734.913664,7990.853285,41.0,752.693750
...,...,...,...,...,...,...,...,...,...,...,...
495,4.0,4.0,0.5,0.4,0.1,18652.00239,28.0,749.266806,9166.329042,37.0,756.565141
496,4.0,4.0,0.1,0.5,0.4,18652.00239,28.0,749.266806,19706.793565,22.0,659.635524
497,4.0,4.0,0.1,0.6,0.3,18652.00239,28.0,749.266806,19706.793565,22.0,659.635524
498,4.0,4.0,0.1,0.7,0.2,18652.00239,28.0,749.266806,14640.719436,21.0,749.623549
